In [1]:
import sys
sys.path.append('./lib/')

In [2]:
from pymongo import MongoClient
import spacy

from extract_biographies import extract_biographies
from generate_sample_collection import copy_sample
from extract_first_sentence import FirstSentenceExtractor
from vectorizer import Vectorizer
from query import Searcher

In [3]:
SPACY_MODEL = 'en_vectors_web_lg'

In [4]:
extract_biographies(collection_name='bios', limit=100)

inserting batch (max 998)...


In [5]:
MongoClient().who.sample_bios.remove({})
copy_sample('bios', 'sample_bios', 10)

/Users/lu/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.


In [6]:
FirstSentenceExtractor().extract('sample_bios', new_field_name='first_sentence')

insertation count: 0


In [7]:
sample_bios = MongoClient().who.sample_bios
next(sample_bios.find({})).keys()

dict_keys(['_id', 'name', 'bio', 'first_sentence'])

In [8]:
next(sample_bios.find({}))['first_sentence']

'Aristotle  was an ancient Greek philosopher and scientist born in the city of Stagira, Chalkidiki, in the north of Classical Greece.'

In [9]:
nlp = spacy.load(SPACY_MODEL)

In [10]:
Vectorizer(nlp).vectorize_text(
    collection_name='sample_bios',
    text_field_name='first_sentence',
    vector_field_name='fresh_new_vectorization'
)

insertation count: 0


In [11]:
next(sample_bios.find({}))['fresh_new_vectorization'][:9]

[0.1327292025089264,
 0.1331862360239029,
 0.014599960297346115,
 -0.11252431571483612,
 0.1476694941520691,
 0.07800917327404022,
 -0.029186425730586052,
 -0.0027071249205619097,
 -0.00013017773744650185]

In [12]:
next(MongoClient().who.sample_bios.find({})).keys()

dict_keys(['_id', 'name', 'bio', 'first_sentence', 'fresh_new_vectorization'])

In [13]:
Searcher(nlp, collection_name='sample_bios', vector_field_name='fresh_new_vectorization').query('king', limit=10)

generating vector similarity maps...


['Paul Castellano',
 'Matthew Hale (jurist)',
 'Sergei Bodrov Jr.',
 'Joseph Gurney Cannon',
 'Ibn al-Shaykh al-Libi',
 'Aristotle',
 'Pope Liberius',
 'Giulio Alenio',
 'David Gates',
 'Algernon Blackwood']